In [ ]:
import requests
import os

url = "http://www.ams.usda.gov/mnreports/wa_gr101.txt"
download_folder = "USDA_Grain_Reports"
os.makedirs(download_folder, exist_ok=True)

filename = os.path.join(download_folder, "WA_GR101_latest.txt")

r = requests.get(url)
if r.status_code == 200:
    with open(filename, "wb") as f:
        f.write(r.content)
    print(f"Downloaded latest WA_GR101 report: {filename}")
else:
    print("Failed to download the report")


Downloaded latest WA_GR101 report: USDA_Grain_Reports/WA_GR101_latest.txt


In [39]:
# Path to the downloaded TXT
filename = "USDA_Grain_Reports/WA_GR101_latest.txt"

grains_of_interest = ["CORN", "SOYBEANS", "WHEAT"]
inspection_numbers = {}

with open(filename, "r") as f:
    lines = f.readlines()

for line in lines:
    line = line.strip()
    if not line:
        continue
    parts = line.split()
    if parts[0] in grains_of_interest:
        # Only try to parse if parts[1] is a number (ignore header lines)
        if parts[1].replace(",", "").isdigit():
            current_week_number = int(parts[1].replace(",", ""))
            inspection_numbers[parts[0]] = current_week_number

print("Latest WA_GR101 weekly inspections:")
for grain, number in inspection_numbers.items():
    print(f"{grain}: {number}")


Latest WA_GR101 weekly inspections:
CORN: 1206913
SOYBEANS: 980518
WHEAT: 183305


In [95]:
import os
import time
from datetime import datetime
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ---------------- CONFIG ----------------
BASE_URL = "https://esmis.nal.usda.gov/publication/grains-inspected-export-weekly"
OUTPUT_DIR = "../data/inspections"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Browser-like headers to avoid 403
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}

# ---------------- SETUP SELENIUM ----------------
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

txt_urls = set()  # avoid duplicates

# ---------------- LOOP PAGES 1-5 ----------------
for page_num in range(0, 37):
    page_url = f"{BASE_URL}?page={page_num-1}"  # page=0 is first page
    print(f"Loading page {page_num} ...")
    driver.get(page_url)
    time.sleep(5)  # wait for JS content

    links_elements = driver.find_elements("xpath", "//a[contains(text(), 'TXT')]")
    print(f"Found {len(links_elements)} TXT links on page {page_num}.")

    for elem in links_elements:
        href = elem.get_attribute("href")
        link_text = elem.text.strip()
        if href and "WA_GR101.TXT" in href:
            txt_urls.add(href)

driver.quit()
print(f"Total WA_GR101.TXT links found: {len(txt_urls)}")

# ---------------- DOWNLOAD FILES ----------------
for url in txt_urls:
    parts = url.split("/")
    folder_part = "_".join(parts[-3:-1])
    filename = parts[-1]
    safe_filename = f"{folder_part}_{filename}"
    output_path = os.path.join(OUTPUT_DIR, safe_filename)

    print(f"Downloading {url} -> {safe_filename} ...")
    try:
        r = requests.get(url, headers=HEADERS)
        r.raise_for_status()
        with open(output_path, "wb") as f:
            f.write(r.content)
        print("Saved successfully.")
    except requests.HTTPError as e:
        print(f"Failed to download {url}: {e}")

print("Done! All WA_GR101.TXT files from pages 1-37 are saved in:", OUTPUT_DIR)


Loading page 0 ...
Found 11 TXT links on page 0.
Loading page 1 ...
Found 11 TXT links on page 1.
Loading page 2 ...
Found 11 TXT links on page 2.
Loading page 3 ...
Found 11 TXT links on page 3.
Loading page 4 ...
Found 11 TXT links on page 4.
Loading page 5 ...
Found 11 TXT links on page 5.
Loading page 6 ...
Found 11 TXT links on page 6.
Loading page 7 ...
Found 11 TXT links on page 7.
Loading page 8 ...
Found 11 TXT links on page 8.
Loading page 9 ...
Found 11 TXT links on page 9.
Loading page 10 ...
Found 11 TXT links on page 10.
Loading page 11 ...
Found 11 TXT links on page 11.
Loading page 12 ...
Found 11 TXT links on page 12.
Loading page 13 ...
Found 11 TXT links on page 13.
Loading page 14 ...
Found 11 TXT links on page 14.
Loading page 15 ...
Found 11 TXT links on page 15.
Loading page 16 ...
Found 11 TXT links on page 16.
Loading page 17 ...
Found 11 TXT links on page 17.
Loading page 18 ...
Found 11 TXT links on page 18.
Loading page 19 ...
Found 11 TXT links on page 19.


In [101]:
import os
import pandas as pd
import re

# ---------------- CONFIG ----------------
DATA_DIR = "../data/inspections"
OUTPUT_CSV = "../data/inspections/weekly_exports.csv"
GRAINS_OF_INTEREST = ["CORN", "SOYBEANS", "WHEAT"]

# ---------------- HELPER FUNCTION ----------------
def parse_file(filepath):
    result = {}
    try:
        with open(filepath, "r", encoding="cp1252", errors="ignore") as f:
            text = f.read()
    except Exception as e:
        print(f"Failed to read {filepath}: {e}")
        return None

    # ---------------- Extract week ending date ----------------
    date_match = re.search(r"REPORTED IN WEEK ENDING\s+([A-Z]{3} \d{2}, \d{4})", text)
    if not date_match:
        print(f"No week ending date found in {filepath}, skipping...")
        return None
    week_ending_date = pd.to_datetime(date_match.group(1))
    result["week_ending_date"] = week_ending_date

    # ---------------- Extract grains ----------------
    for grain in GRAINS_OF_INTEREST:
        # Look for a line starting with the grain name
        pattern = rf"^{grain}\s+([\d,]+)"
        match = re.search(pattern, text, re.MULTILINE)
        if match:
            try:
                result[grain] = int(match.group(1).replace(",", ""))
            except ValueError:
                result[grain] = 0
        else:
            result[grain] = 0

    return result

# ---------------- SCAN DIRECTORY ----------------
records = []
for filename in os.listdir(DATA_DIR):
    if filename.upper().endswith(".TXT"):
        filepath = os.path.join(DATA_DIR, filename)
        data = parse_file(filepath)
        if data:
            data["source_file"] = filename
            records.append(data)

# ---------------- BUILD DATAFRAME ----------------
df = pd.DataFrame(records)
df[GRAINS_OF_INTEREST] = df[GRAINS_OF_INTEREST].fillna(0)
df = df.sort_values("week_ending_date").reset_index(drop=True)

# ---------------- SAVE CSV ----------------
df.to_csv(OUTPUT_CSV, index=False)
print(f"Processed {len(df)} weeks and saved DataFrame to {OUTPUT_CSV}")
print(df.head())



No week ending date found in ../data/inspections/7s75dn385_02871499k_WA_GR101.TXT, skipping...
No week ending date found in ../data/inspections/9w032885t_st74cx562_WA_GR101.TXT, skipping...
No week ending date found in ../data/inspections/nc580t76b_df65vg480_WA_GR101.TXT, skipping...
No week ending date found in ../data/inspections/6q182s16v_wm117w530_WA_GR101.TXT, skipping...
Processed 355 weeks and saved DataFrame to ../data/inspections/weekly_exports.csv
  week_ending_date     CORN  SOYBEANS   WHEAT  \
0       2018-12-20   996098    651181  543126   
1       2018-12-27   913797    677679  376281   
2       2019-01-03   501541    673172  260134   
3       2019-01-10  1013682   1085251  545804   
4       2019-01-17  1108119   1110713  516643   

                        source_file  
0  zs25xd601_ht24wp864_WA_GR101.TXT  
1  m900nz807_d504rq53g_WA_GR101.TXT  
2  7h149v10w_ff3659758_WA_GR101.TXT  
3  sf268991d_xg94hv293_WA_GR101.TXT  
4  c534fv264_1v53k3121_WA_GR101.TXT  
